In [1]:
# Import Python Packages
import numpy
import glob
import scipy.io
from netCDF4 import Dataset
import matplotlib.pyplot as mp
import matplotlib.cm as cm

In [2]:
#####################################
###   User-specified Section      ###
###                for plotting   ###
#####################################

## Model output directory & filename
MODEL='R2+TMIv7'
res='0.25'

## Number of Regions
# Use region 1 to NUMBER_OF_REGIONS in the mask
NUMBER_OF_REGIONS=4
REGION_STR=['WPac','EPac','Atl','Ind']

## Use 1:tave, or 2:qsat as Bulk Tropospheric Temperature Measure 
BULK_TROPOSPHERIC_TEMPERATURE_MEASURE=1
TAVE_VAR='tave'
QSAT_INT_VAR='qsat_int'

## Directory & Filename for saving binned results (netCDF4)
BIN_OUTPUT_DIR='/home/kyh/MDTF_onset_diag_v06/var_data/convecTransStat_R2TMIv7/'
BIN_OUTPUT_FILENAME='convecTransStat_R2TMIv7r1_200206_201405_res='+res+'_fillNrCWV'
if BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==1:
    BIN_OUTPUT_FILENAME+='_'+TAVE_VAR+'.mat'
elif BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==2:
    BIN_OUTPUT_FILENAME+='_'+QSAT_INT_VAR+'.mat'

NC_OUTPUT_FILENAME='convecTransStat_R2TMIv7r1_200206_201405_res='+res+'_fillNrCWV'
if BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==1:
    NC_OUTPUT_FILENAME+='_'+TAVE_VAR+'.nc'
elif BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==2:
    NC_OUTPUT_FILENAME+='_'+QSAT_INT_VAR+'.nc'

In [3]:
bin_obs_filename=BIN_OUTPUT_DIR+BIN_OUTPUT_FILENAME
matfile=scipy.io.loadmat(bin_obs_filename)
cwv_bin_center=matfile['cwv']
P0=matfile['P0'].astype(float)
P1=matfile['P1']
P2=matfile['P2']
PE=matfile['PE'].astype(float)
PRECIP_THRESHOLD=matfile['PRECIP_THRESHOLD'][0,0]
if BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==1:
    tave_bin_center=matfile['tave']
    Q0=matfile['Q0'].astype(float)
    Q1=matfile['Q1']
elif BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==2:
    qsat_int_bin_center=matfile['qsat_int']

In [4]:
# Save Binning Results
bin_output_netcdf=Dataset(BIN_OUTPUT_DIR+NC_OUTPUT_FILENAME,"w",format="NETCDF4")
            
bin_output_netcdf.description="Convective Transition Statistics compiled using \
NCEP/DOE Reanalysis-2 6-hourly Atmospheric Temperature Field and TRMM Microwave Imager \
(TMI) CWV Retrieval Product processed by Remote Sensing System (RSS) algorithm v7r1"
bin_output_netcdf.source="Convective Onset Statistics Diagnostic Package \
- as part of the NOAA Model Diagnostic Task Force (MDTF) effort"
bin_output_netcdf.PRECIP_THRESHOLD=PRECIP_THRESHOLD

region=bin_output_netcdf.createDimension("region",NUMBER_OF_REGIONS)
reg=bin_output_netcdf.createVariable("region",numpy.float64,("region",),zlib=True)
reg=numpy.arange(1,NUMBER_OF_REGIONS+1)

cwv=bin_output_netcdf.createDimension("cwv",len(cwv_bin_center))
prw=bin_output_netcdf.createVariable("cwv",numpy.float64,("cwv",),zlib=True)
prw.units="mm"
prw[:]=cwv_bin_center

if (BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==1):
    tave=bin_output_netcdf.createDimension(TAVE_VAR,len(tave_bin_center))
    temp=bin_output_netcdf.createVariable("tave",numpy.float64,(TAVE_VAR,),zlib=True)
    temp.units="K"
    temp[:]=tave_bin_center

    p0=bin_output_netcdf.createVariable("P0",numpy.float64,("region","cwv",TAVE_VAR),zlib=True)
    p0[:,:,:]=P0

    p1=bin_output_netcdf.createVariable("P1",numpy.float64,("region","cwv",TAVE_VAR),zlib=True)
    p1.units="mm/hr"
    p1[:,:,:]=P1

    p2=bin_output_netcdf.createVariable("P2",numpy.float64,("region","cwv",TAVE_VAR),zlib=True)
    p2.units="mm^2/hr^2"
    p2[:,:,:]=P2

    pe=bin_output_netcdf.createVariable("PE",numpy.float64,("region","cwv",TAVE_VAR),zlib=True)
    pe[:,:,:]=PE

    q0=bin_output_netcdf.createVariable("Q0",numpy.float64,("region",TAVE_VAR),zlib=True)
    q0[:,:]=Q0

    q1=bin_output_netcdf.createVariable("Q1",numpy.float64,("region",TAVE_VAR),zlib=True)
    q1.units="mm"
    q1[:,:]=Q1

elif (BULK_TROPOSPHERIC_TEMPERATURE_MEASURE==2):
    qsat_int=bin_output_netcdf.createDimension(QSAT_INT_VAR,len(qsat_int_bin_center))
    temp=bin_output_netcdf.createVariable("qsat_int",numpy.float64,(QSAT_INT_VAR,),zlib=True)
    temp.units="mm"
    temp[:]=qsat_int_bin_center

    p0=bin_output_netcdf.createVariable("P0",numpy.float64,("region","cwv",QSAT_INT_VAR),zlib=True)
    p0[:,:,:]=P0

    p1=bin_output_netcdf.createVariable("P1",numpy.float64,("region","cwv",QSAT_INT_VAR),zlib=True)
    p1.units="mm/hr"
    p1[:,:,:]=P1

    p2=bin_output_netcdf.createVariable("P2",numpy.float64,("region","cwv",QSAT_INT_VAR),zlib=True)
    p2.units="mm^2/hr^2"
    p2[:,:,:]=P2

    pe=bin_output_netcdf.createVariable("PE",numpy.float64,("region","cwv",QSAT_INT_VAR),zlib=True)
    pe[:,:,:]=PE

bin_output_netcdf.close()